# 2025 CITS4012 Project 2

## 1. Preprocessing

### 1.1. Import Libraries
Libraries are imported for the entire project.

*Note: Just add to this list as needed*

In [4]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

### 1.2. Retrieve Data
First, we need to retrieve the train/val/test datafiles from the Google Drive.

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
train_fileid = "1YEOo5vd8DXXUCf1FXCR1D3PxWR9XxQKv"
val_fileid = "15FEgtzzTVDMQcNVMgwIwqoAJeF9vmtrX"
test_fileid = "179nwaOvdkZ3ogsBaTSJvpZEIjq20uiG-"

train_file = drive.CreateFile({'id': train_fileid})
val_file = drive.CreateFile({'id': val_fileid})
test_file = drive.CreateFile({'id': test_fileid})

train_file.GetContentFile('train.json')
val_file.GetContentFile('val.json')
test_file.GetContentFile('test.json')

### 1.2. Read in the Data
We now read in each dataset, splitting the input data and target data. The data shapes and a sample of the training data are shown below.

In [20]:
train = pd.read_json('train.json')
X_train = train[['premise', 'hypothesis']]
y_train = train['label']

val = pd.read_json('val.json')
X_val = val[['premise', 'hypothesis']]
y_val = val['label']

test = pd.read_json('test.json')
X_test = test[['premise', 'hypothesis']]
y_test = test['label']

print(f"Train shape: {X_train.shape}")
print(f"Val shape: {X_val.shape}")
print(f"Test shape: {X_test.shape}")
train.head()

Train shape: (23088, 2)
Val shape: (1304, 2)
Test shape: (2126, 2)


,premise,hypothesis,label
0,Pluto rotates once on its axis every 6.39 Eart...,Earth rotates on its axis once times in one day.,neutral
1,---Glenn =====================================...,Earth rotates on its axis once times in one day.,entails
2,geysers - periodic gush of hot water at the su...,The surface of the sun is much hotter than alm...,neutral
3,Facts: Liquid water droplets can be changed in...,Evaporation is responsible for changing liquid...,entails
4,"By comparison, the earth rotates on its axis o...",Earth rotates on its axis once times in one day.,entails


### 1.3. Data preprocessing
As seen above, the data contains characters that requires cleaning before passing it to the models.